# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from config import gkey

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Load csv that was exported to DF
colnames = ['City', 'Latitude', 'Longitude', 'Max_Temp', 'Humidity', 'Cloudy', 'Wind', 'Country', 'Date']
data = pd.read_csv('../output_data/City_Weather.csv', names=colnames)

data_df = data.drop([data.index[0]])

# Drop NaN
data_df.dropna(inplace = True) 



### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Store latitude and longitude in locations
locations = data_df[["Latitude", "Longitude"]].astype(float)

# Add Humid as Weights
humid = data_df["Humidity"].astype(float)

# Configure gmaps with API key
gmaps.configure(api_key=gkey)

# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humid, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig                 

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
# Set filters on ideal vacation city

# convert max temp and clouds to numbers
data_df['Max_Temp'] = pd.to_numeric(data_df['Max_Temp'])
data_df['Cloudy'] = pd.to_numeric(data_df['Cloudy'])
data_df['Humidity'] = pd.to_numeric(data_df['Humidity'])

# temp between 75-90
mask = (data_df['Max_Temp'] > 75) & (data_df['Max_Temp'] < 90)
vac_city1 = data_df.loc[mask]

#Clouds less than 25
vac_city2 = vac_city1.loc[(vac_city1['Cloudy'] < 25)]

#humidity less than 60
vac_city3 = vac_city2.loc[(vac_city2['Humidity'] < 30)]
vac_city3

,City,Latitude,Longitude,Max_Temp,Humidity,Cloudy,Wind,Country,Date
6,mokshan,53.44,44.61,86.00,26.0,0.0,3.31,RU,1592659053.0
105,el paso,31.76,-106.49,78.80,9.0,20.0,4.7,US,1592658890.0
135,ufa,54.77,56.04,80.60,28.0,0.0,17.9,RU,1592659070.0
251,east london,-33.02,27.91,80.60,18.0,0.0,11.41,ZA,1592659085.0
312,karatau,43.17,70.47,80.31,20.0,0.0,15.61,KZ,1592659093.0
317,tatawin,32.93,10.45,89.60,22.0,0.0,12.75,TN,1592659094.0
367,mehran,33.12,46.16,86.00,7.0,0.0,13.87,IR,1592659101.0
467,shihezi,44.3,86.03,87.15,14.0,7.0,7.7,CN,1592658927.0
489,manzanares,39.0,-3.37,89.60,17.0,0.0,8.05,ES,1592659117.0
523,lisakovsk,52.54,62.49,76.12,29.0,0.0,17.52,KZ,1592659044.0


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
# Set dataframe to hotel_df
hotel_df = vac_city3

hotel_df["Hotel Name"] =""

# Store 'Lat' and 'Lng' into  locations 
locations = hotel_df[["Latitude", "Longitude"]].astype(float)
hotel_df


C:\Users\Matt\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,City,Latitude,Longitude,Max_Temp,Humidity,Cloudy,Wind,Country,Date,Hotel Name
6,mokshan,53.44,44.61,86.00,26.0,0.0,3.31,RU,1592659053.0,
105,el paso,31.76,-106.49,78.80,9.0,20.0,4.7,US,1592658890.0,
135,ufa,54.77,56.04,80.60,28.0,0.0,17.9,RU,1592659070.0,
251,east london,-33.02,27.91,80.60,18.0,0.0,11.41,ZA,1592659085.0,
312,karatau,43.17,70.47,80.31,20.0,0.0,15.61,KZ,1592659093.0,
317,tatawin,32.93,10.45,89.60,22.0,0.0,12.75,TN,1592659094.0,
367,mehran,33.12,46.16,86.00,7.0,0.0,13.87,IR,1592659101.0,
467,shihezi,44.3,86.03,87.15,14.0,7.0,7.7,CN,1592658927.0,
489,manzanares,39.0,-3.37,89.60,17.0,0.0,8.05,ES,1592659117.0,
523,lisakovsk,52.54,62.49,76.12,29.0,0.0,17.52,KZ,1592659044.0,


In [6]:
# Put in locations and get Hotel Name
# geocoordinates

target_radius = 5000
target_type = "lodging"

# set up a parameters dictionary
params = {
    "radius": target_radius,
    "type": target_type,
    "key": gkey
}

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

for index, row in hotel_df.iterrows():

    # get restaurant type from df
    location = f"{row['Latitude']}, {row['Longitude']}"

    # add keyword to params dict
    params['location'] = location

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {location}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']

    try:
        print(f"Closest {location} hotel is {results[0]['name']}.")
        
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

# run a request using our params dictionary
response = requests.get(base_url, params=params)

# convert response to json
places_data = response.json()

# Print the name and address of the first restaurant that appears
print(places_data["results"][0]["name"])

#######################################################################
#Add column to dataframe for hotel name
#hotel_df['Hotel Name'] = 
hotel_df


Retrieving Results for Index 6: 53.44, 44.61.
Closest 53.44, 44.61 hotel is AZK Next.
Retrieving Results for Index 105: 31.76, -106.49.


C:\Users\Matt\anaconda3\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Closest 31.76, -106.49 hotel is Hilton Garden Inn El Paso / University.
Retrieving Results for Index 135: 54.77, 56.04.
Closest 54.77, 56.04 hotel is Azimut Hotel Ufa.
Retrieving Results for Index 251: -33.02, 27.91.
Closest -33.02, 27.91 hotel is Tu Casa.
Retrieving Results for Index 312: 43.17, 70.47.
Closest 43.17, 70.47 hotel is Gor.bol'nitsa.
Retrieving Results for Index 317: 32.93, 10.45.
Closest 32.93, 10.45 hotel is Sangho Privilege Tataouine.
Retrieving Results for Index 367: 33.12, 46.16.
Closest 33.12, 46.16 hotel is زائرسرای نور.
Retrieving Results for Index 467: 44.3, 86.03.
Closest 44.3, 86.03 hotel is Home Inn Shihezi Dongwu Road Century Square.
Retrieving Results for Index 489: 39.0, -3.37.
Closest 39.0, -3.37 hotel is Parador de Manzanares.
Retrieving Results for Index 523: 52.54, 62.49.
Closest 52.54, 62.49 hotel is Gostinitsa "Kontinent".
Retrieving Results for Index 525: 41.83, 48.49.
Closest 41.83, 48.49 hotel is Samur.
Retrieving Results for Index 545: 42.8, 93.45

,City,Latitude,Longitude,Max_Temp,Humidity,Cloudy,Wind,Country,Date,Hotel Name
6,mokshan,53.44,44.61,86.00,26.0,0.0,3.31,RU,1592659053.0,AZK Next
105,el paso,31.76,-106.49,78.80,9.0,20.0,4.7,US,1592658890.0,Hilton Garden Inn El Paso / University
135,ufa,54.77,56.04,80.60,28.0,0.0,17.9,RU,1592659070.0,Azimut Hotel Ufa
251,east london,-33.02,27.91,80.60,18.0,0.0,11.41,ZA,1592659085.0,Tu Casa
312,karatau,43.17,70.47,80.31,20.0,0.0,15.61,KZ,1592659093.0,Gor.bol'nitsa
317,tatawin,32.93,10.45,89.60,22.0,0.0,12.75,TN,1592659094.0,Sangho Privilege Tataouine
367,mehran,33.12,46.16,86.00,7.0,0.0,13.87,IR,1592659101.0,زائرسرای نور
467,shihezi,44.3,86.03,87.15,14.0,7.0,7.7,CN,1592658927.0,Home Inn Shihezi Dongwu Road Century Square
489,manzanares,39.0,-3.37,89.60,17.0,0.0,8.05,ES,1592659117.0,Parador de Manzanares
523,lisakovsk,52.54,62.49,76.12,29.0,0.0,17.52,KZ,1592659044.0,"Gostinitsa ""Kontinent"""


In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]



In [24]:
# Convert to floats
locations["Latitude"] = locations["Latitude"].apply(lambda x: float(x))
locations["Longitude"] = locations["Longitude"].apply(lambda x: float(x))

C:\Users\Matt\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\Matt\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [30]:
# Add marker layer ontop of heat map
hotel_layer = gmaps.marker_layer(
    locations, 
#     fill_color='rgba(0, 150, 0, 0.4)',
#     stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=[info_box_template.format(**row) for index, row in hotel_df.iterrows()]
)
fig.add_layer(hotel_layer)

fig

# Display figure


Figure(layout=FigureLayout(height='420px'))